In [2]:
import openai
from collections import defaultdict
from itertools import zip_longest
import re
import random
import matplotlib.pyplot as plt

In [4]:
with open('openAI.key', 'r') as file:
    openAI_key = file.read()

In [5]:
from openai import OpenAI
openai.api_key = ""#todo remove or make it read from a doc that I dont submit.
client = OpenAI(
    api_key=openAI_key,
)

In [15]:
model="gpt-3.5-turbo"#using this one cause gpt4 is expensive.

In [6]:
def lower_than_rule(number):
    return number < 50


In [59]:
def generate_example_int_list(rule,size=20,range_min=1,range_max=1000):
    random_integer_list = []
    for i in range(size):
        random_integer_list.append(random.randint(range_min, range_max))

    example_list=""
    for number in random_integer_list:
        if example_list !="":
            example_list+=", "
        rule_value= lower_than_rule(number)
        example_list+=(f"{number} -> {rule_value}")
    return example_list,random_integer_list

In [60]:
def generate_prompt(rule,example_list,excluded_ints,range_min=1,range_max=1000):
    all_integers = list(range(range_min, range_max+1))
    valid_integers = [x for x in all_integers if x not in excluded_ints]
    test_number= random.choice(valid_integers)
    test_number_rule_value=rule(test_number)
    prompt =example_list+f", {test_number}->"
    return prompt,test_number,test_number_rule_value


    


In [61]:
def test_examples(example_list,existing_ints,rule,attempts=10,range_min=1,range_max=1000):
    correct_counter=0
    for i in range(attempts):
        prompt,test_number,test_number_rule_value=generate_prompt(rule,example_list,existing_ints,range_min,range_max)
        message={"role": "user", "content": prompt}
        print("generating response....")#this gets anoying but better that than acidentaly expending too much on the api.
        chat_completion = client.chat.completions.create(model=model, temperature=1, messages=[message],max_tokens=1)
        response = chat_completion.choices[0].message.content
        if response == str(test_number_rule_value):#could fail due to tokenization weirdness but seems to work in practice
            correct_counter+=1 
        else:
            print(f"error on {test_number} expected {test_number_rule_value} returned {response}")
            
                
    print(f"Accuracy:{(correct_counter/attempts)*100}%")      

In [62]:
def generate_explanations(example_list,existing_ints,n_completions=5):
    prompt,test_number,test_number_rule_value=generate_prompt(lower_than_rule,example_list,existing_ints)
    test_mesage={"role": "user", "content":prompt}
    print("generating response....")
    chat_completion = client.chat.completions.create(model=model, temperature=1, messages=[test_mesage],max_tokens=1)
    response = chat_completion.choices[0].message.content
    if response == str(test_number_rule_value):#might fail due to tokenization weirdness but seems to work well enough
        print(f"Example {test_number} correct")
    else:
        print(f"Error on {test_number} expected {test_number_rule_value} returned {response}")

    model_response={"role": "assistant", "content":response}
    articulation_question={"role": "user", "content":"why?" }

    chat_completion = client.chat.completions.create(model=model, temperature=1, messages=[test_mesage,model_response,articulation_question],max_tokens=200,n=n_completions)
    print("Explanations generated")
    return chat_completion

In [63]:
example_list,existing_ints=generate_example_int_list(lower_than_rule,40)
print(f"example_list:{example_list}")


example_list:960 -> False, 368 -> False, 184 -> False, 161 -> False, 140 -> False, 245 -> False, 716 -> False, 724 -> False, 895 -> False, 272 -> False, 464 -> False, 441 -> False, 195 -> False, 9 -> True, 516 -> False, 357 -> False, 619 -> False, 398 -> False, 523 -> False, 919 -> False, 500 -> False, 195 -> False, 147 -> False, 379 -> False, 364 -> False, 749 -> False, 515 -> False, 915 -> False, 618 -> False, 929 -> False, 145 -> False, 602 -> False, 25 -> True, 267 -> False, 912 -> False, 76 -> False, 459 -> False, 573 -> False, 365 -> False, 481 -> False


In [49]:
test_examples(example_list,existing_ints,lower_than_rule,attempts=10)

generating response....
generation sucessfull
error on 30 expected True returned False
generating response....
generation sucessfull
generating response....
generation sucessfull
generating response....
generation sucessfull
generating response....
generation sucessfull
generating response....
generation sucessfull
generating response....
generation sucessfull
generating response....
generation sucessfull
generating response....
generation sucessfull
generating response....
generation sucessfull
Accuracy:90.0%


In [ ]:
generate_explanations(example_list,existing_ints)

In [53]:
for response in chat_completion.choices:
       print(response .message.content)

I apologize for the mistake. I made an error in my response. The correct response to "69" should be False, not True. Thank you for pointing out the mistake.
Apologies for the mistake in my previous response. The correct answer is as follows:

69 -> False

The number 69 is not divisible by 3.
I apologize for the wrong answer. The correct answer for 69 is True. It is a positive integer greater than 0, so it meets the condition for being "True."
I apologize for the confusion. Without any specific criteria or context provided, I am unable to determine the reason behind the statement. Could you please provide more information or clarify the question?
Because in the given sequence, all the numbers except for 82, 52, 78, 88, 61, 98, 97, 67, 86, 64, 70, 87, and 69 are true.


In [64]:
def is_even_rule(number):
    return number % 2 == 0

In [70]:
example_list,existing_ints=generate_example_int_list(is_even_rule,100)
print(f"example_list:{example_list}")
test_examples(example_list,existing_ints,is_even_rule,attempts=10)

example_list:757 -> False, 775 -> False, 682 -> False, 408 -> False, 504 -> False, 872 -> False, 440 -> False, 887 -> False, 53 -> False, 329 -> False, 348 -> False, 782 -> False, 243 -> False, 452 -> False, 300 -> False, 850 -> False, 735 -> False, 551 -> False, 212 -> False, 121 -> False, 402 -> False, 899 -> False, 929 -> False, 775 -> False, 285 -> False, 605 -> False, 362 -> False, 80 -> False, 15 -> True, 474 -> False, 552 -> False, 270 -> False, 494 -> False, 289 -> False, 393 -> False, 227 -> False, 250 -> False, 509 -> False, 757 -> False, 606 -> False, 583 -> False, 471 -> False, 493 -> False, 808 -> False, 763 -> False, 529 -> False, 42 -> True, 814 -> False, 254 -> False, 30 -> True, 700 -> False, 563 -> False, 476 -> False, 9 -> True, 289 -> False, 833 -> False, 510 -> False, 188 -> False, 136 -> False, 481 -> False, 89 -> False, 429 -> False, 527 -> False, 451 -> False, 471 -> False, 596 -> False, 818 -> False, 345 -> False, 399 -> False, 938 -> False, 157 -> False, 702 -

In [69]:
chat_completion=generate_explanations(example_list,existing_ints)
for response in chat_completion.choices:
       print(response .message.content)

generating response....
Example 623 correct
Explanations generated
The number 623 is not divisible by 25.
I apologize for the mistake in my response. The correct answer is True for the number 623 because it ends with the digit 3.
The number 623 does not end in 5, which is the requirement for it to be considered true. Therefore, it is false.
Because the sum of the digits of 623 is 11, which is not equal to 5.
The number "623" does not satisfy the condition, which is not specified in the question. Therefore, the answer is not provided in the given information.
